In [1]:
# !pip install transformers datasets==2.21.0 tokenizers seqeval scikit-learn num2words -q

In [ ]:
import re
import numpy as np 
import pandas as pd
import json
from num2words import num2words

from datasets import load_dataset, load_metric
import transformers
from transformers import BertTokenizerFast, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer, pipeline 

from sklearn.model_selection import train_test_split

In [3]:
random_seed = 1
transformers.set_seed(seed=random_seed)

## Transform NLP Data

In [ ]:
# df_nlp = pd.read_json('data/nlp.jsonl', lines=True)
# df_nlp = pd.read_json('../data/nlp.jsonl', lines=True).iloc[:1000]
df_nlp = pd.read_json('../data/nlp.jsonl', lines=True).sample(n=1000)

df_nlp['tokens'] = df_nlp['transcript'].apply(lambda x: re.findall(r'[.?]|\w+', x))
df_nlp['tool'] = df_nlp['tool'].apply(lambda x: re.findall(r'[.?]|\w+', x))
df_nlp['heading'] = df_nlp['heading'].apply(lambda x: list(map(lambda y: num2words(int(y)), list(str(x).zfill(3)))))
df_nlp['target'] = df_nlp['target'].apply(lambda x: re.findall(r'[.?]|\w+', x))

def map_tags(lst, items:str):
	dic = {}
	begin = ''
	inbet = ''
	if items == 'tool':
		begin = 1 #'B-TOL'
		inbet = 2 #'I-TOL'
	elif items == 'heading':
		begin = 3 #'B-HDG'
		inbet = 4 #'I-HDG'
	elif items == 'target':
		begin = 5 #'B-TAR'
		inbet = 6 # 'I-TAR'
	dic[lst[0]] = begin
	for i in range(1, len(lst)):
		dic[lst[i]] = inbet
	return dic

df_nlp['tool'] = df_nlp['tool'].apply(lambda x: map_tags(x, 'tool'))
df_nlp['heading'] = df_nlp['heading'].apply(lambda x: map_tags(x, 'heading'))
df_nlp['target'] = df_nlp['target'].apply(lambda x: map_tags(x, 'target'))
df_nlp = df_nlp[['key', 'tokens', 'tool', 'heading', 'target']]
df_nlp = df_nlp.rename(columns={'key':'id'})

def map_num(row):
	curr_dict = row['tool'].copy()
	curr_dict.update(row['heading'])
	curr_dict.update(row['target'])	
	return [0 if curr_dict.get(i) is None else curr_dict.get(i) for i in row['tokens']]

df_nlp['ner_tags'] = df_nlp.apply(map_num, axis=1)
df_nlp

In [5]:
label_list = ['O', 'B-TOL', 'I-TOL', 'B-HDG', 'I-HDG', 'B-TAR', 'I-TAR']

## Load NLP Data

In [ ]:
train, test = train_test_split(df_nlp, test_size=0.2, random_state=random_seed)
train, val = train_test_split(train, test_size=0.125, random_state=random_seed) # 0.125 x 0.8 = 0.1
train.shape, val.shape, test.shape

In [7]:
train.to_json('../data/nlp_train.jsonl', orient='records', lines=True)
test.to_json('../data/nlp_test.jsonl', orient='records', lines=True)
val.to_json('../data/nlp_val.jsonl', orient='records', lines=True)

In [ ]:
_nlp_data = {
	'train' : '../data/nlp_train.jsonl',
	'validation' : '../data/nlp_val.jsonl',
	'test' : '../data/nlp_test.jsonl'
}

nlp_data =  load_dataset('json', data_files=_nlp_data) 
nlp_data

## Tokenization

In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased") 

In [10]:
def tokenize_and_align_labels(examples, label_all_tokens=True): 
    """
    Function to tokenize and align labels with respect to the tokens. This function is specifically designed for
    Named Entity Recognition (NER) tasks where alignment of the labels is necessary after tokenization.

    Parameters:
    examples (dict): A dictionary containing the tokens and the corresponding NER tags.
                     - "tokens": list of words in a sentence.
                     - "ner_tags": list of corresponding entity tags for each word.
                     
    label_all_tokens (bool): A flag to indicate whether all tokens should have labels. 
                             If False, only the first token of a word will have a label, 
                             the other tokens (subwords) corresponding to the same word will be assigned -100.

    Returns:
    tokenized_inputs (dict): A dictionary containing the tokenized inputs and the corresponding labels aligned with the tokens.
    """
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True) 
    labels = [] 
    for i, label in enumerate(examples["ner_tags"]): 
        word_ids = tokenized_inputs.word_ids(batch_index=i) 
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token. 
        previous_word_idx = None 
        label_ids = []
        # Special tokens like `<s>` and `<\s>` are originally mapped to None 
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids: 
            if word_idx is None: 
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token                 
                label_ids.append(label[word_idx]) 
            else: 
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100) 
                # mask the subword representations after the first subword
                 
            previous_word_idx = word_idx 
        labels.append(label_ids) 
    tokenized_inputs["labels"] = labels 
    return tokenized_inputs 

### Add Labels

In [ ]:
tokenized_datasets = nlp_data.map(tokenize_and_align_labels, batched=True)

# Model Training

In [12]:
def compute_metrics(eval_preds): 
    """
    Function to compute the evaluation metrics for Named Entity Recognition (NER) tasks.
    The function computes precision, recall, F1 score and accuracy.

    Parameters:
    eval_preds (tuple): A tuple containing the predicted logits and the true labels.

    Returns:
    A dictionary containing the precision, recall, F1 score and accuracy.
    """
    pred_logits, labels = eval_preds 
    
    pred_logits = np.argmax(pred_logits, axis=2) 
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax
    
    # We remove all the values where the label is -100
    predictions = [ 
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100] 
        for prediction, label in zip(pred_logits, labels) 
    ] 
    
    true_labels = [ 
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100] 
       for prediction, label in zip(pred_logits, labels) 
   ] 
    results = metric.compute(predictions=predictions, references=true_labels) 
    return { 
   "precision": results["overall_precision"], 
   "recall": results["overall_recall"], 
   "f1": results["overall_f1"], 
  "accuracy": results["overall_accuracy"], 
  } 

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=7)

args = TrainingArguments( 
	"test-ner",
	evaluation_strategy = "epoch", 
	learning_rate=2e-5, 
	per_device_train_batch_size=16, 
	per_device_eval_batch_size=16, 
	num_train_epochs=3, 
	weight_decay=0.01, 
) 

data_collator = DataCollatorForTokenClassification(tokenizer) 

metric = load_metric("seqeval") 

trainer = Trainer( 
    model, 
	args, 
	train_dataset=tokenized_datasets["train"], 
	eval_dataset=tokenized_datasets["validation"], 
	data_collator=data_collator, 
	tokenizer=tokenizer, 
	compute_metrics=compute_metrics 
) 

In [ ]:
trainer.train() 

## Save Model

In [ ]:
model.save_pretrained("ner_model")
tokenizer.save_pretrained("tokenizer")

In [16]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

config = json.load(open("ner_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id

# Model Prediction

In [17]:
json.dump(config, open("ner_model/config.json","w"))

In [18]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("ner_model")

In [ ]:
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)


example = "Shoot down the car, red it is not, but blue is, and black as well as white, 1 six two, javelin is a missiles at the ready"

ner_results = nlp(example)

for i in ner_results:
	print(i)